In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
import sys, os
sys.path.append(os.path.abspath(os.path.join("..")))

In [5]:
from model.model_uniform import UniformGaborCNN

In [6]:
# check if the model actually has expected number of wavelets after initialization
net = UniformGaborCNN(use_dilation=False)
x = torch.randn(2,1,96,96)

with torch.no_grad():
    z = net.act(net.norm(net.gabor(x)))
print("Post-Gabor shape:", z.shape, " expected channels:", net.gabor.number_of_wavelets)

Post-Gabor shape: torch.Size([2, 40, 96, 96])  expected channels: 40


In [7]:
# check if the weight buffer requires grad. it should not, because of the implementation in gaborfilterfrequency.py
def reset_weight_buffer(g):
    with torch.no_grad():
        if 'weight' in g._parameters:
            del g._parameters['weight']
        if 'weight' in g._buffers:
            del g._buffers['weight']
        H, W = g.kernel_size
        buf = torch.empty(g.out_channels, H, W, device=g.sigma.device, dtype=g.sigma.dtype)
        g.register_buffer('weight', buf)  

reset_weight_buffer(net.gabor)

print('buffer requires_grad:', getattr(net.gabor.weight, 'requires_grad', None)) 

buffer requires_grad: False


In [8]:
# check if the sigma and k's change with one simple forward

test_model = UniformGaborCNN(image_size=(96,96), num_classes=14).cuda() 
test_model.train()

x = torch.randn(8, 1, 96, 96, device='cuda')
y = torch.randint(0, 14, (8,), device='cuda')

opt = torch.optim.Adam(test_model.parameters(), lr=1e-3)
opt.zero_grad()

logits = test_model(x)
loss = torch.nn.functional.cross_entropy(logits, y)
loss.backward()

sig_grad = getattr(test_model.gabor.sigma, 'grad', None)
freq_grad = getattr(test_model.gabor.frequency, 'grad', None)
print('‖grad sigma‖:', float(sig_grad.norm()) if sig_grad is not None else None)
print('‖grad k‖    :', float(freq_grad.norm()) if freq_grad is not None else None)

‖grad sigma‖: 0.01282531674951315
‖grad k‖    : 1.101483702659607


In [9]:
print("is param:", "weight" in net.gabor._parameters) # this should not be a parameter for the optimizer 
print("is buffer:", "weight" in net.gabor._buffers) # instead it is registered as a buffer
print("requires_grad:", getattr(net.gabor.weight, "requires_grad", None)) # therefore it should not require grad

is param: False
is buffer: True
requires_grad: False
